In [9]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders.base import BaseLoader
from langchain.schema import Document
import json
import re
import numpy as np
from typing import List
from langchain.text_splitter import TextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from dotenv import load_dotenv
import os
import pickle
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains.query_constructor.base import AttributeInfo, get_query_constructor_prompt
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [10]:
load_dotenv() 
openai_api_key = os.getenv("OPENAI_API_KEY")


In [11]:
class JSONLLoader(BaseLoader):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self):
        documents = []
        seq_num = 1
        
        with open(self.file_path, 'r', encoding='utf-8') as file:
            for line in file:
                data = json.loads(line)
                doc = Document(
                    page_content=data['content'],
                    metadata={
                        'docid': data['docid'],
                        'src': data.get('src', ''),  # 'src' 필드가 없을 경우 빈 문자열 사용
                        'source': self.file_path,
                        'seq_num': seq_num,
                    }
                )
                documents.append(doc)
                seq_num += 1
        
        return documents

file_path = "/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl"
loader = JSONLLoader(file_path)
documents = loader.load()

print(f"문서의 수: {len(documents)}")

문서의 수: 4272


In [12]:
vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())

In [13]:
# OpenAI LLM 초기화 
llm = OpenAI(temperature=0)

In [14]:
# 유사도 검색 테스트 
query = "나무의 분류에 대해 조사해 보기 위한 방법은?"

In [15]:
# Standalone Query Generator 프롬프트 템플릿
standalone_query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""질문 query를 요약하려고 합니다. 핵심내용을 포함한 주제를 출력해주세요. 하나의 구문으로 출력해주세요. 
    아래는 예시입니다. 
    
    원래의 질문: "금성이 밝게 보이는 이유가 뭐야?"
    생성할 standalone_query: "금성 밝기 원인"

    원래의 질문: {question}
    독립적인 질문:
    """
)

# Standalone Query Generator 체인 생성
standalone_query_chain = LLMChain(llm=llm, prompt=standalone_query_prompt)

# Standalone Query 생성
standalone_query = standalone_query_chain.run(query)

print(f"Original Query: {query}")
print(f"Standalone Query: {standalone_query}")

Original Query: 나무의 분류에 대해 조사해 보기 위한 방법은?
Standalone Query: 
    나무 분류 조사 방법


In [17]:
# K개만 검색하여 결과를 반환
K = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": K})
search_result = retriever.get_relevant_documents(standalone_query)

# 결과 출력
for i, doc in enumerate(search_result, 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content[:100]}...")  # 처음 100자만 출력
    print(f"메타데이터: {doc.metadata}")
    print("---")



문서 1:
내용: 한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러...
메타데이터: {'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'source': '/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl', 'seq_num': 2199}
---

문서 2:
내용: 고든 여사의 수업은 자연과학에 관한 것입니다. 이 수업에서는 단풍나무에 대해 공부합니다. 단풍나무는 가을에 아름다운 빨간색으로 변하는 나무로 유명합니다. 학생들은 이 나무의 특성을...
메타데이터: {'docid': 'b730a81a-3903-42ca-9633-88b0ebb9eb42', 'src': 'ko_ai2_arc__ARC_Challenge__train', 'source': '/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl', 'seq_num': 474}
---

문서 3:
내용: 학생이 다른 토양 유형에서 식물의 성장을 연구했습니다. 연구에서는 날짜와 식물의 높이, 그리고 토양 유형을 기록하였습니다. 이러한 데이터를 정리하는 가장 좋은 방법은 날짜와 식물의...
메타데이터: {'docid': 'a153c822-be9f-4346-8558-34365ed7b4f0', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'source': '/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl', 'seq_num': 1414}
---


In [44]:
science_check_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
다음 질문이 지식을 묻는 내용인지 판단해주세요:

질문: {question}

이 질문이 상식이나 지식을 묻는 것(물리, 화학, 생물, 지구과학, 천문학, 컴퓨터공학, 소프트웨어 등)이라면 "True"라고만 답변하고, 그렇지 않다면 "False"라고만 답변해주세요.

예시:
1. "물의 화학식은 무엇인가요?" -> True
2. "지구의 자전 주기는 얼마인가요?" -> True
3. "오늘 기분이 아주 좋아" -> False
4. "너는 어쩜 모르는게 없구나?" -> False
5. "니가 대답을 잘해줘서 너무 고마워!" -> False
6. "v파이

답변:
"""
)

science_check_chain = LLMChain(llm=llm, prompt=science_check_prompt)

In [45]:
def is_science_related(query):
    science_check_result = science_check_chain.run(query).strip().lower()
    return science_check_result == "true"

In [46]:
def answer_question(query):
    # 과학 상식 관련 질문인지 확인
    if not is_science_related(query):
        return {
            "answer": "",
            "standalone_query": "",
            "is_science_related": False,
            "topk": [],
            "references": []
        }


    # Standalone Query 생성
    standalone_query = standalone_query_chain.run(query)

    # standalone_query가 과학 상식과 관련 있는지 확인
    standalone_query_science_related = is_science_related(standalone_query)

    # 검색 수행 (K개 검색)
    K = 3
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": K})
    search_result = retriever.get_relevant_documents(standalone_query)

    # RAG 프롬프트 가져오기
    rag_prompt = hub.pull("rlm/rag-prompt")

    # 첫 번째 문서의 내용만 사용
    context = search_result[0].page_content if search_result else ""

    # LLM 체인 생성
    llm = OpenAI(temperature=0)
    rag_chain = LLMChain(llm=llm, prompt=rag_prompt)

    # 답변 생성 (첫 번째 문서만 참고)
    answer = rag_chain.run(context=context, question=query)
    
    standalone_query = standalone_query_chain.run(query).strip('"')  # 따옴표 제거

    # topk 및 references 정보 추출
    topk = [doc.metadata.get('docid') for doc in search_result]
    
    # 이미 검색된 문서의 내용을 사용
    references = [
        {
            "score": doc.metadata.get('score', 0),
            "content": doc.page_content
        }
        for doc in search_result
    ]


    return {
        "standalone_query": standalone_query,
        "is_science_related": standalone_query_science_related,
        "topk": topk,
        "answer": answer,
        "references": references
    }


In [47]:
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}')
            print(f'Standalone Query: {response["standalone_query"]}')
            print(f'Is Science Related: {response["is_science_related"]}\n')

            output = {
                "eval_id": j["eval_id"],
                "standalone_query": response["standalone_query"],
                # "is_science_related": response["is_science_related"],
                "topk": response["topk"],
                "answer": response["answer"],
                "references": response["references"]
            }
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [48]:
eval_rag("/data/ephemeral/home/upstage-ai-final-ir2/upstage-ai-final-ir2/HM/data/eval.jsonl", "sample_submission_8.csv")


Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer:  이 학생은 나무의 분류를 위해 성장 속도, 온도 범위, 크기, 잎과 꽃의 유사성 등을 고려하고 있습니다. 이러한 특징들은 나무의 속을 파악하는 데 중요한 역할을 합니다. 나무의 분류는 생물 분류학에서 중요한 기준 중 하나이며, 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다.
Standalone Query: 나무 분류 조사 방법
Is Science Related: False

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 
Standalone Query: 
Is Science Related: False

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 
Standalone Query: 
Is Science Related: False

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 
Standalone Query: 
Is Science Related: False

Test 4
Question: [{'role': 'user', 'content': 'Dmitri Ivanovsky가 누구야?'}]
Answer: 
Standalone Query: 
Is Science Related: False

Test 5
Question: [{'role': 'user', 'content': '피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?

KeyboardInterrupt: 